## 1. Importação de bibliotecas e carregamento dos dados

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


In [ ]:

# Carregar o dataset
df = pd.read_csv('maquinas.csv')  # ajuste o caminho se necessário
df.head()


## 2. Análise exploratória e limpeza de dados

In [ ]:

# Verificar tipos e valores ausentes
print(df.dtypes)
print(df.isnull().sum())

# Exibir estatísticas básicas
df.describe(include='all')


## 3. Tratamento de outliers com mediana

In [ ]:

df_out = df.copy()
numericas = df_out.select_dtypes(include=['int64', 'float64']).columns

for col in numericas:
    Q1 = df_out[col].quantile(0.25)
    Q3 = df_out[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    mediana = df_out[col].median()

    df_out[col] = np.where(
        (df_out[col] < limite_inferior) | (df_out[col] > limite_superior),
        mediana,
        df_out[col]
    )


## 4. Codificação de variáveis categóricas

In [ ]:

df_codificado = df_out.copy()
categoricas = df_codificado.select_dtypes(include='object').columns
label_encoders = {}

for col in categoricas:
    le = LabelEncoder()
    df_codificado[col] = le.fit_transform(df_codificado[col])
    label_encoders[col] = le


## 5. Normalização das variáveis numéricas

In [ ]:

scaler = MinMaxScaler()
df_normalizado = df_codificado.copy()
df_normalizado[numericas] = pd.DataFrame(
    scaler.fit_transform(df_normalizado[numericas]),
    columns=numericas,
    index=df_normalizado.index
)


## 6. Separação entre variáveis preditoras e alvo

In [ ]:

# Para regressão
X_reg = df_normalizado.drop(columns=['preco', 'classe', 'status'])
y_reg = df_normalizado['preco']

# Para classificação
X_class = df_normalizado.drop(columns=['status', 'classe', 'preco'])
y_class = df_normalizado['status']


## 7. Modelos de regressão e classificação

In [ ]:

Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

modelo_rf = RandomForestRegressor(n_estimators=100, random_state=42)
modelo_rf.fit(Xr_train, yr_train)
yr_pred = modelo_rf.predict(Xr_test)

rmse = np.sqrt(mean_squared_error(yr_test, yr_pred))
r2 = r2_score(yr_test, yr_pred)

print("Random Forest Regressor:")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")


In [ ]:

Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

modelo_rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf_clf.fit(Xc_train, yc_train)
yc_pred = modelo_rf_clf.predict(Xc_test)

acc = accuracy_score(yc_test, yc_pred)
print("Random Forest Classifier:")
print(f"Acurácia: {acc:.2f}")
print("Matriz de Confusão:")
print(confusion_matrix(yc_test, yc_pred))
print("
Relatório de Classificação:")
print(classification_report(yc_test, yc_pred))


## 8. Importância das variáveis preditoras

In [ ]:

importancias_reg = modelo_rf.feature_importances_
nomes_reg = X_reg.columns
indices_reg = np.argsort(importancias_reg)[::-1]

plt.figure(figsize=(10, 6))
plt.title("Importância das Variáveis - Regressão (preço)")
plt.bar(range(len(importancias_reg)), importancias_reg[indices_reg])
plt.xticks(range(len(importancias_reg)), nomes_reg[indices_reg], rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:

importancias_clf = modelo_rf_clf.feature_importances_
nomes_clf = X_class.columns
indices_clf = np.argsort(importancias_clf)[::-1]

plt.figure(figsize=(10, 6))
plt.title("Importância das Variáveis - Classificação (status)")
plt.bar(range(len(importancias_clf)), importancias_clf[indices_clf])
plt.xticks(range(len(importancias_clf)), nomes_clf[indices_clf], rotation=45, ha='right')
plt.tight_layout()
plt.show()


## 📊 Visualizações Extras

In [ ]:

# Boxplot das variáveis numéricas
plt.figure(figsize=(15, 8))
df_normalizado.boxplot(rot=45)
plt.title("Boxplot das Variáveis Numéricas (normalizadas)")
plt.show()


In [ ]:

# Pairplot com amostra para não travar
sns.pairplot(df_normalizado.sample(n=200, random_state=42))
plt.suptitle("Pairplot com Amostra", y=1.02)
plt.show()


## 💾 Exportação do DataFrame Tratado

In [ ]:

# Exportar o DataFrame tratado
df_normalizado.to_csv("maquinas_tratado.csv", index=False)
print("Arquivo 'maquinas_tratado.csv' exportado com sucesso.")


In [ ]:

import joblib

# Exportar modelo de regressão (previsão de preço)
joblib.dump(modelo_rf, "modelo_preco.pkl")
print("Modelo salvo como modelo_preco.pkl")
